In [5]:
# Parameters

#! ВСЕ РАЗДЕЛИТЬ НА ПЛОТНОСТЬ ЧТОБЫ БЫЛО ЗАЕБИСЬ!
# todo проверить вывод уравнений (все что связано с плотностью)

# Mesh parameters
MESH_DENSITY = 8
INLET_VELOCITY = 1.5 # m/s

# Simulation parameters
SIMULATION_TIME = 2
TIME_STEP_LENGTH = 10**-4
N_TIME_STEPS = int(SIMULATION_TIME/TIME_STEP_LENGTH) 

# Fluid properties
DENSITY = 998 # kg/m^3
VISCOSITY = 8.9 * 10**-4 # Pa*s (N*s/m^2) 
THERMAL_CONDUCTIVITY = 0.598 # W/(m * K) 
SPECIFIC_HEAT_CAPACITY = 4184 # J/(kg * K) 

print('(Derived constants)')
print(f'Reybolds Number (Re): {DENSITY*INLET_VELOCITY*0.41/VISCOSITY:.5f}')
print(f'Kinematic viscosity (nu) [m^2/s]: {VISCOSITY / DENSITY}',)
print(f'Thermal diffusivity (kappa) [m^2/s]: {THERMAL_CONDUCTIVITY/(DENSITY*SPECIFIC_HEAT_CAPACITY)}',)

print('(Simulation)')
print(f'Time step [s]: {TIME_STEP_LENGTH}')
print(f'Total time [s]: {SIMULATION_TIME}')
print(f'Iterations: {N_TIME_STEPS}')
print(f'Mesh density: {MESH_DENSITY}')


(Derived constants)
Reybolds Number (Re): 689629.21348
Kinematic viscosity (nu) [m^2/s]: 8.917835671342686e-07
Thermal diffusivity (kappa) [m^2/s]: 1.4321185391816137e-07
(Simulation)
Time step [s]: 0.0001
Total time [s]: 2
Iterations: 20000
Mesh density: 8


In [4]:
!pip3 install fenics

# Define mesh
import fenics as fe
import mshr as ms
import matplotlib.pyplot as plt

# Create the mesh using Constructive solid geometry (CSG)
# box = ms.Rectangle(fe.Point(0.0, 0.0), fe.Point(1.0, 1.0))
# hole = ms.Rectangle(fe.Point(0.2,0.4),fe.Point(0.8,0.6)) # 
# domain = box 

channel = ms.Rectangle(
    fe.Point(0.0,0.0),
    fe.Point(2.2,0.41)
)
cylinder = ms.Circle(fe.Point(0.2,0.2),0.05)

domain = channel - cylinder
mesh = ms.generate_mesh(domain, MESH_DENSITY)

# If loading mesh from file
# mesh = fe.Mesh('path.xml')

fig, ax = plt.subplots(dpi=200)
ax.set_xlabel('x')
ax.set_ylabel('y')
fe.plot(mesh, title='Mesh',linewidth=0.5)
plt.show()

print('Number of cell elements:',mesh.num_cells())
fe.set_log_active(False) # suspend FEniCS output

ModuleNotFoundError: No module named 'fenics'

In [27]:
# Form compiler options
fe.parameters["form_compiler"]["optimize"]     = True
fe.parameters["form_compiler"]["cpp_optimize"] = True
#fe.parameters["form_compiler"]["representation"] = "quadrature"

In [28]:
# Define the function spaces
# Taylor-Hood Elements. The order of the function space for the pressure has
# to be one order lower than for the velocity
velocity_function_space = fe.VectorFunctionSpace(mesh, "Lagrange", 2)
pressure_function_space = fe.FunctionSpace(mesh, "Lagrange", 1)
heat_function_space = fe.FunctionSpace(mesh,'Lagrange', 1)

# Define the trial and test functions
u = fe.TrialFunction(velocity_function_space)
v = fe.TestFunction(velocity_function_space)
p = fe.TrialFunction(pressure_function_space)
q = fe.TestFunction(pressure_function_space)
t = fe.TrialFunction(heat_function_space)
w = fe.TestFunction(heat_function_space)

In [29]:
# Define the solution fields involved
u_prev = fe.Function(velocity_function_space) # velocity at previous time step, used for time integration
u_tent = fe.Function(velocity_function_space) # velocity before pressure correction
u_next = fe.Function(velocity_function_space,name='Velocity') # velocity after pressure correction, at current time step
p_next = fe.Function(pressure_function_space,name='Pressure') # pressure at current time step
t_prev = fe.Function(heat_function_space)
t_next = fe.Function(heat_function_space,name='Heat')


In [30]:
# Define the Boundary Condition
def walls(x, on_boundary): return on_boundary
def top_boundary(x, on_boundary): return on_boundary and fe.near(x[1], 0.41)
def bottom_boundary(x, on_boundary): return on_boundary and fe.near(x[1], 0.0)
def left_boundary(x, on_boundary): return on_boundary and fe.near(x[0], 0.0)
def right_boundary(x, on_boundary): return on_boundary and fe.near(x[0], 2.2)

inflow_1 = fe.Constant((INLET_VELOCITY, 0.0))
noslip = fe.Constant((0.0, 0.0))
inflow_cylinder_problem = fe.Expression(
    ('4.0*1.5*x[1]*(0.41 - x[1]) / pow(0.41, 2)', '0'),
    degree = 2
)

velocity_boundary_conditions = [
    fe.DirichletBC(velocity_function_space, noslip, walls),
    fe.DirichletBC(velocity_function_space, inflow_cylinder_problem, left_boundary)
]

pressure_boundary_conditions = [
    fe.DirichletBC(pressure_function_space, 0.0, right_boundary)
]

heat_boundary_conditions = [
    fe.DirichletBC(heat_function_space, 200, left_boundary),
    fe.DirichletBC(heat_function_space, 0, right_boundary)
]


In [31]:
# Set initial conditions
# t_prev.interpolate(
#     fe.Expression(
#         '60 * (0.5*(1 - tanh(8*(sqrt(pow(x[0]-0.5, 2) + pow(x[1]-0.5, 2)) - 0.1))))',
#      degree=1)
# )
# u_prev.interpolate(
#     fe.Expression(
#     ('sin(x[1])', 'cos(x[0])'),
#     degree=2
#     )
# )

In [32]:
# Define weak forms
dt = fe.Constant(TIME_STEP_LENGTH)
rho = fe.Constant(DENSITY)
nu = fe.Constant(VISCOSITY / DENSITY)
kappa = fe.Constant(THERMAL_CONDUCTIVITY/(DENSITY*SPECIFIC_HEAT_CAPACITY))

# Weak form of the momentum equation
momentum_residuum = ( #? force term
    (1.0 / dt) * fe.inner(u - u_prev, v) * fe.dx
    +
    fe.inner(fe.grad(u_prev) * u_prev, v) * fe.dx # Advection term
    +
    nu * fe.inner(fe.grad(u), fe.grad(v)) * fe.dx) # Diffusion term


# Weak form of the pressure poisson problem
pressure_poisson_residuum = (
    fe.inner(fe.grad(p), fe.grad(q)) * fe.dx
    +
    (rho / dt) * fe.div(u_tent) * q * fe.dx
)

# Weak form of the velocity update equation
velocity_update_residuum = (
    fe.inner(u, v) * fe.dx
    -
    fe.inner(u_tent, v) * fe.dx
    +
    (dt/rho) * fe.inner(fe.grad(p_next), v) * fe.dx)

# Weak form of heat transfer equation
heat_residuum = (
    (1.0 / dt) * fe.inner(t - t_prev, w) * fe.dx
    +
    fe.dot(u_next,fe.grad(t_prev)) * w *fe.dx # Advection
    +
    kappa * fe.inner(fe.grad(t_prev), fe.grad(w)) * fe.dx # Diffusion term
)


In [33]:
# Assemble system matricies
# used to solve linear (Ax=b) systems 
momentum_lhs = fe.lhs(momentum_residuum)
momentum_rhs = fe.rhs(momentum_residuum)
momentum_A = fe.assemble(momentum_lhs)

pressure_poisson_lhs = fe.lhs(pressure_poisson_residuum)
pressure_poisson_rhs = fe.rhs(pressure_poisson_residuum)
pressure_poisson_A = fe.assemble(pressure_poisson_lhs)

velocity_update_lhs = fe.lhs(velocity_update_residuum)
velocity_update_rhs = fe.rhs(velocity_update_residuum)
velocity_update_A = fe.assemble(velocity_update_lhs)

heat_lhs = fe.lhs(heat_residuum)
heat_rhs = fe.rhs(heat_residuum)
heat_A = fe.assemble(heat_lhs)


Calling FFC just-in-time (JIT) compiler, this may take some time.


In [34]:
# Time loop
from tqdm import tqdm  # for progress bar

# Use amg preconditioner if available
prec = "amg" if fe.has_krylov_solver_preconditioner("amg") else "default"

# Creating file for saving results
with fe.XDMFFile(fe.MPI.comm_world, 'testing_SI_units_new_heat.xdmf') as file:
    file.parameters.update(
        {
            "functions_share_mesh": True,
            "rewrite_function_mesh": False
        })

    # Saving initial states
    file.write(u_next, 0)
    file.write(p_next, 0)
    file.write(t_next, 0)

    # Simulation
    for iter in tqdm(range(N_TIME_STEPS)):

        # (1) Solve for tentative velocity
        momentum_b = fe.assemble(momentum_rhs)
        [bc.apply(momentum_A, momentum_b)
            for bc in velocity_boundary_conditions]
        fe.solve(
            momentum_A,
            u_tent.vector(),
            momentum_b,
            "gmres",
            "default",
        )

        # (2) Solve for the pressure
        pressure_poisson_b = fe.assemble(pressure_poisson_rhs)
        [bc.apply(pressure_poisson_A, pressure_poisson_b)
            for bc in pressure_boundary_conditions]
        fe.solve(
            pressure_poisson_A,
            p_next.vector(),
            pressure_poisson_b,
            "cg",
            prec,
        )

        # (3) Correct the velocities to be incompressible
        velocity_update_b = fe.assemble(velocity_update_rhs)
        [bc.apply(velocity_update_A, velocity_update_b)
            for bc in velocity_boundary_conditions]
        fe.solve(
            velocity_update_A,
            u_next.vector(),
            velocity_update_b,
            "gmres",
            "default",
        )

        # (4) Solve heat advection-diffusion equation
        # fe.solve(heat_lhs==heat_rhs,t_next,heat_boundary_conditions)
        # heat_b = fe.assemble(heat_rhs)
        # [bc.apply(heat_A, heat_b) for bc in heat_boundary_conditions]
        # fe.solve(
        #     heat_A,
        #     t_next.vector(),
        #     heat_b,
        #     "cg",
        #     prec,
        # )

        # Save to file
        file.write(u_next, iter*TIME_STEP_LENGTH)
        file.write(p_next, iter*TIME_STEP_LENGTH)
        #file.write(t_next, iter*TIME_STEP_LENGTH)

        # Advance in time
        t_prev.assign(t_next)
        u_prev.assign(u_next)


 14%|█▍        | 2763/20000 [03:16<20:23, 14.09it/s]  


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to solve linear system using PETSc Krylov solver.
*** Reason:  Solution failed to converge in 0 iterations (PETSc reason DIVERGED_NANORINF, residual norm ||r|| = inf).
*** Where:   This error was encountered inside PETScKrylovSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  3ea2183fbfe7277de9f16cbe1a9ffaab133ba1fa
*** -------------------------------------------------------------------------


In [ ]:
# Plot results
import matplotlib.pyplot as plt

plt.figure(dpi=150)

pressure_plot = fe.plot(p_next,
                        cmap='jet',
                        levels=50)
plt.colorbar(pressure_plot,
             label='Pressure',
             location='bottom')

velocity_plot = fe.plot(u_next,
                        edgecolor='black',
                        linewidth=0.5,
                        minshaft=1,
                        minlength=1.5,
                        title='Velocity/Pressure',)
plt.colorbar(velocity_plot,
             label='Velocity,m/s',
             location='bottom')
fe.plot(mesh, linewidth=0.1)
plt.show()


In [ ]:
# Plot heat
import cmocean

plt.figure(figsize=(14, 14), dpi=150)
heat_plot = fe.plot(
    t_next, 
    cmap=cmocean.cm.thermal,
    levels=50)
#fe.plot(t_next,mode='contour',levels=10,cmap='coolwarm')
plt.colorbar(heat_plot, label='Heat, $\degree C$',location='bottom')
fe.plot(mesh, linewidth=0.1)
fe.plot(u_next,edgecolor='white',cmap=cmocean.cm.speed,alpha=0.1)
plt.show()

In [ ]:
# Plot curl
import cmocean

plt.figure(figsize=(14, 14), dpi=150)
curl_plot = fe.plot(fe.curl(u_next), 
                    cmap=cmocean.cm.balance,
                    levels=10,
                    title='$\operatorname{curl} \mathbf{u}$')
plt.colorbar(curl_plot, label='curl',location='bottom')
fe.plot(mesh, linewidth=0.1)
fe.plot(u_next,cmap=cmocean.cm.speed,alpha = 0.1)
plt.show()